# Programming a neural network layer

[Keras](https://keras.io) is a high-level deep-learning framework building on top of [TensorFlow](https://www.tensorflow.org). These frameworks follow the _symbol-to-symbol derivatives_ approach, i.e. automatically derive a computational graph to calculate derivatives. You just need to declare your inputs as TensorFlow variables and use TensorFlow operations on them to compute the forward pass.  

## Task 6.1

Work through the [Keras tutorial on custom layers](https://keras.io/guides/making_new_layers_and_models_via_subclassing) to learn how to create your own neural network layer.  
Create a custom Keras layer that computes Gaussian basis functions, i.e. a layer that maps an input vector $\mathbf x \in \mathbb R^n$ onto an output vector $\mathbf y = f(\mathbf x) \in \mathbb R^m$ as follows:
\begin{align}
  f: \mathbf x \in \mathbb R^n \mapsto \left[w_i \exp\left(-\frac{\|\mathbf x - \boldsymbol\mu_i\|^2}{\sigma_i^2}\right)\right]_{i=1..m} \in \mathbb R^m
\end{align}

Instead of projecting an input $\mathbf x$ onto a weight vector $\mathbf w$ as the standard neuron function $f(\mathbf x) = \sigma(\mathbf w \cdot \mathbf x + b)$ does, the Gaussian basis function becomes active (with weight $w_i$) for all inputs $\mathbf x$ close to a prototype $\boldsymbol \mu_i$. This activation quickly decays with increasing distance of $\mathbf x$ to $\boldsymbol \mu_i$. The parameter $\sigma_i$ controls the width of the Gaussian, i.e. the size of the active region.

For efficient tensor-based operations you need to correctly _broadcast_ the tensors for the difference operation: TensorFlow will pass an input matrix of shape `(batch size, input dim)` for $\mathbf X$, while you will have a matrix of centers $\boldsymbol \mu$ of shape `(input dim, #units)`. To correctly [broadcast](https://numpy.org/doc/stable/user/basics.broadcasting.html) them together, you will need Keras' [`expand_dims()`](https://www.tensorflow.org/api_docs/python/tf/keras/backend/expand_dims) function to extend $\mathbf X$'s shape to `(batch size, input dim, 1)`:

In [ ]:
import numpy
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K

class RBFLayer(keras.layers.Layer):
    # TODO: Add initializer parameters to init
    def __init__(self, **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        # TODO: Do something with the additional parameters

    def build(self, input_shape):
        super(RBFLayer, self).build(input_shape)
        # TODO: Initialize properties depending on `input_shape`

    def call(self, X):
        # TODO: Process input and return
        return X


class InitCentersRandom(keras.initializers.Initializer):
    """ Initializer to initialize centers of RBF network from random samples of the data set."""

    def __init__(self, X):
        self.X = X

    def __call__(self, shape, dtype=None):
        # TODO: Generate and return initialization
        pass


### dummy example to compute diffs between data X and centers mu
X = tf.ones((3, 5))  # input tensor X with batch dimension 3 and data dim N=5
mu = tf.ones((5, 2))  # tensor mu with data dim N=5 and 2 units
diffs = K.expand_dims(X) - mu  # diffs tensor: 3 x 5 x 2
print(X.shape, mu.shape, diffs.shape)

## Task 6.2

Compare the performance of such a Gaussian basis function layer with that of a standard [`Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense) layer on the MNIST dataset.  
Hint: Utilize existing tutorials on setting up your first MNIST MLP with Keras, e.g. https://www.tensorflow.org/guide/keras/train_and_evaluate.

To achieve decent performance, you want to:
- Initialize the centers $\boldsymbol \mu_i$ from random data samples $\mathbf x$ (create a custom [initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/Initializer))
- Initialize $\sigma_i$ to the typical in-class distance between data points.  
  Use [`scipy.spatial.distance_matrix`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance_matrix.html) to compute this statistics on a random selection of your input data.  
  (Doing it on the full dataset will probably exhaust your memory.)
- Initialize $w_i = 1$

Questions:
- How many parameters each of those networks have?
- Which network trains faster / easier?